In [1]:
import geopandas as gpd
import pygeos as pygeos

from geopy import distance
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from simpledbf import Dbf5

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [2]:
def calc_dist(df):
    x, y = df['centroid_x'], df['centroid_y']
    
    x_nn, y_nn = df['nn_centroid_x'], df['nn_centroid_y']
    
    return distance.distance((y,x), (y_nn, x_nn)).meters

In [3]:
#roads_shp = "portugal-latest-free.shp 2/gis_osm_roads_free_1.shp"                  # ORIGINAL DA DSSG PT    
roads_shp = "../../dados_osm/portugal-latest-free.shp/gis_osm_roads_free_1.shp"     # Caminho ajustado ao projeto no local


In [4]:
roads = gpd.read_file(roads_shp)

In [5]:
freedom_roads = roads[(roads.name.str.contains("25 de Abril")) | (roads.name.str.contains("Liberdade")) | (roads.name.str.contains("Salgueiro Maia")) | (roads.name.str.contains("Capitães de Abril"))]

In [6]:

df2 = freedom_roads[['osm_id', 'name', 'geometry']]
df2 = df2.rename(columns={'name': 'rua'})

In [7]:
df2.head(3)

,osm_id,rua,geometry
224,4453413,Avenida da Liberdade,"LINESTRING (-9.02960 39.75781, -9.02954 39.757..."
237,4472168,Avenida da Liberdade,"LINESTRING (-9.03221 39.75903, -9.03204 39.758..."
430,5044364,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90207 32.6..."


In [8]:
#concelhos = gpd.read_file('concelhos-shapefile/concelhos.shp', SHAPE_RESTORE_SHX='YES')                  # ORIGINAL DA DSSG PT
concelhos = gpd.read_file('../../dados_osm/concelhos-shapefile/concelhos.shp',SHAPE_RESTORE_SHX='YES')    # Caminho ajustado ao projeto no local

concelhos = concelhos.rename(columns={'NAME_2': 'concelho'})

df2['centroid_json'] = df2.geometry.apply(lambda x: x.centroid)
df2 = df2.sjoin(concelhos[['concelho', 'geometry']], how="left")


In [9]:
#freguesias = gpd.read_file('cont-aad-caop2017/Cont_AAD_CAOP2017.shp', SHAPE_RESTORE_SHX='YES')                  # ORIGINAL DA DSSG PT
freguesias = gpd.read_file('../../dados_osm/Cont_AAD_CAOP2021/Cont_AAD_CAOP2021.shp', SHAPE_RESTORE_SHX='YES')   # Caminho ajustado ao projeto no local


In [10]:
df2 = df2.drop(columns=['index_right'])

In [11]:
freguesias.to_crs(epsg=4326, inplace=True)

In [12]:
df2 = df2.sjoin(freguesias[['Freguesia', 'geometry']], how="left")

In [13]:
# Spatial join leading to some duplicates
df2 = df2.drop_duplicates('osm_id')

In [14]:
df2['centroid_x'] = df2.geometry.apply(lambda x: x.centroid.x)
df2['centroid_y'] = df2.geometry.apply(lambda x: x.centroid.y)

In [15]:
df2.Freguesia = df2.Freguesia.fillna('')

In [16]:
df2.head(3)

,osm_id,rua,geometry,centroid_json,concelho,index_right,Freguesia,centroid_x,centroid_y
224,4453413,Avenida da Liberdade,"LINESTRING (-9.02960 39.75781, -9.02954 39.757...",POINT (-9.02876 39.75742),Marinha Grande,1174.0,Marinha Grande,-9.028758,39.757420
237,4472168,Avenida da Liberdade,"LINESTRING (-9.03221 39.75903, -9.03204 39.758...",POINT (-9.03106 39.75850),Marinha Grande,1174.0,Marinha Grande,-9.031063,39.758498
430,5044364,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90207 32.6...",POINT (-16.90205 32.65254),Funchal,NaN,,-16.902049,32.652540


In [17]:
# m = folium.Map(location=[39.9, -8.22])

#for i, row in df2.iterrows():
#    folium.Marker([row['centroid_y'], row['centroid_x']]).add_to(m)


In [18]:
df2['rua_freguesia_concelho'] = df2['rua'] + "_" +  df2['Freguesia'] + "_" + df2['concelho']
df2

,osm_id,rua,geometry,centroid_json,concelho,index_right,Freguesia,centroid_x,centroid_y,rua_freguesia_concelho
224,4453413,Avenida da Liberdade,"LINESTRING (-9.02960 39.75781, -9.02954 39.757...",POINT (-9.02876 39.75742),Marinha Grande,1174.0,Marinha Grande,-9.028758,39.757420,Avenida da Liberdade_Marinha Grande_Marinha Gr...
237,4472168,Avenida da Liberdade,"LINESTRING (-9.03221 39.75903, -9.03204 39.758...",POINT (-9.03106 39.75850),Marinha Grande,1174.0,Marinha Grande,-9.031063,39.758498,Avenida da Liberdade_Marinha Grande_Marinha Gr...
430,5044364,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90207 32.6...",POINT (-16.90205 32.65254),Funchal,NaN,,-16.902049,32.652540,Via 25 de Abril__Funchal
436,5045625,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90227 32.6...",POINT (-16.90228 32.65270),Funchal,NaN,,-16.902285,32.652700,Via 25 de Abril__Funchal
966,10073105,Ponte 25 de Abril,"LINESTRING (-9.17801 38.70838, -9.17807 38.708...",POINT (-9.17779 38.69398),Almada,636.0,"União das freguesias de Almada, Cova da Piedad...",-9.177787,38.693978,Ponte 25 de Abril_União das freguesias de Alma...
...,...,...,...,...,...,...,...,...,...,...
1171688,1052145667,Rua da Liberdade,"LINESTRING (-8.41328 37.84876, -8.41450 37.84876)",POINT (-8.41389 37.84876),Odemira,416.0,Vale de Santiago,-8.413888,37.848760,Rua da Liberdade_Vale de Santiago_Odemira
1172199,1052529928,Rua da Liberdade,"LINESTRING (-7.87258 39.17144, -7.87249 39.171...",POINT (-7.87189 39.17201),Avis,996.0,União das freguesias de Benavila e Valongo,-7.871892,39.172011,Rua da Liberdade_União das freguesias de Benav...
1172201,1052529930,Rua 25 de Abril,"LINESTRING (-7.86940 39.17372, -7.86981 39.173...",POINT (-7.87163 39.17204),Avis,996.0,União das freguesias de Benavila e Valongo,-7.871628,39.172043,Rua 25 de Abril_União das freguesias de Benavi...
1172335,1052736975,Travessa 25 de Abril,"LINESTRING (-7.87257 39.10771, -7.87279 39.10716)",POINT (-7.87268 39.10744),Avis,996.0,União das freguesias de Benavila e Valongo,-7.872679,39.107435,Travessa 25 de Abril_União das freguesias de B...


In [19]:
le = LabelEncoder()
df2['unique_index'] = le.fit_transform(df2.rua_freguesia_concelho)
df2 = df2.drop(columns=['centroid_json'])
df2

,osm_id,rua,geometry,concelho,index_right,Freguesia,centroid_x,centroid_y,rua_freguesia_concelho,unique_index
224,4453413,Avenida da Liberdade,"LINESTRING (-9.02960 39.75781, -9.02954 39.757...",Marinha Grande,1174.0,Marinha Grande,-9.028758,39.757420,Avenida da Liberdade_Marinha Grande_Marinha Gr...,170
237,4472168,Avenida da Liberdade,"LINESTRING (-9.03221 39.75903, -9.03204 39.758...",Marinha Grande,1174.0,Marinha Grande,-9.031063,39.758498,Avenida da Liberdade_Marinha Grande_Marinha Gr...,170
430,5044364,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90207 32.6...",Funchal,NaN,,-16.902049,32.652540,Via 25 de Abril__Funchal,1399
436,5045625,Via 25 de Abril,"LINESTRING (-16.90216 32.65268, -16.90227 32.6...",Funchal,NaN,,-16.902285,32.652700,Via 25 de Abril__Funchal,1399
966,10073105,Ponte 25 de Abril,"LINESTRING (-9.17801 38.70838, -9.17807 38.708...",Almada,636.0,"União das freguesias de Almada, Cova da Piedad...",-9.177787,38.693978,Ponte 25 de Abril_União das freguesias de Alma...,341
...,...,...,...,...,...,...,...,...,...,...
1171688,1052145667,Rua da Liberdade,"LINESTRING (-8.41328 37.84876, -8.41450 37.84876)",Odemira,416.0,Vale de Santiago,-8.413888,37.848760,Rua da Liberdade_Vale de Santiago_Odemira,1264
1172199,1052529928,Rua da Liberdade,"LINESTRING (-7.87258 39.17144, -7.87249 39.171...",Avis,996.0,União das freguesias de Benavila e Valongo,-7.871892,39.172011,Rua da Liberdade_União das freguesias de Benav...,1177
1172201,1052529930,Rua 25 de Abril,"LINESTRING (-7.86940 39.17372, -7.86981 39.173...",Avis,996.0,União das freguesias de Benavila e Valongo,-7.871628,39.172043,Rua 25 de Abril_União das freguesias de Benavi...,713
1172335,1052736975,Travessa 25 de Abril,"LINESTRING (-7.87257 39.10771, -7.87279 39.10716)",Avis,996.0,União das freguesias de Benavila e Valongo,-7.872679,39.107435,Travessa 25 de Abril_União das freguesias de B...,1322


In [20]:
df2.to_file('../ruas_abril.geojson', driver='GeoJSON')

C:\Users\ernes\anaconda3\envs\p04_ruas25abril\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
